<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# Импортируем необходимые библиотеки.
! pip install sweetviz
import pandas as pd
import numpy as np
import sweetviz as sv
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

     |████████████████████████████████| 15.1 MB 1.2 MB/s eta 0:00:01


In [2]:
# Импортируем и изучим данные
#df = pd.read_csv('C:\\datasets\insurance.csv')
df = pd.read_csv('/datasets/insurance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# Сделаем расширенный EDA-анализ
report = sv.analyze([df, " EDA-анализ данных"])
report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
# Проверим на наличие дубликатов
df.duplicated().sum()

153

**Выводы**
В датасете 5000 не нулевых записей, 153 дубликата. Оставим их без удаления, т.к. по условию нам нужно защитить данные. Мужчин и женщин - примерно одинаковое количество. Признаки распределены без аномалий, данные готовы для дальнейшей работы.

## Умножение матриц

Для защиты данных клиентов будет использоваться метод умножения исходной матрицы признаков данных Х на случайную квадратную матрицу (обозначим её P), размером по количеству столбцов в исходной.

Тогда формула предсказаний будет иметь вид:
$$
a' = X'w' = (XP)w'
$$
Новая формула обучения:
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
Последовательно раскроем скобки:
$$
w' = ((P^T X^T) XP)^{-1} P^T X^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$ (P^T)^{-1} P^T $ это единичная матрица E , а матрица, умножаясь на единичную - равна себе, поэтому можно сократить запись:
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
$ (X^T X)^{-1}$ - не будем раскрывать, так как для неквадратных матриц ($X$ и $X^T$) обратных матриц не существует
Так как $ (X^T X)^{-1} X^T y = w $, поэтому получаем следующую формулу:
$$
w' = P^{-1} w
$$
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w' = a'
$$


Проверим, не ухудшается ли качество модели в результате преобразования:

In [6]:
# Разделим данные на признаки и целевой признак: features и target
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [7]:
# Создадим класс Regression и напишем 2 метода fit и predict.
class Regression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv((X.T @ X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        print(w)
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [8]:
# Создадим модель и обучим её. Найдем её предсказания на обучающей выборке. Рассчитаем метрику R2.
model = Regression()
model.fit(features, target)
predictions = model.predict(features)
r2_score(target, predictions)

[-9.38235504e-01  7.92580543e-03  3.57083050e-02 -1.70080492e-07
 -1.35676623e-02]


0.42494550286668

In [9]:
# Создадим случайную матрицу
random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
random_matrix

array([[ 0.519866  ,  0.48317294,  1.2930181 ,  0.02682421],
       [-0.99581739, -0.80547415, -0.70593036, -0.93506952],
       [-1.71105073, -0.74933498,  1.4569403 , -0.76785483],
       [ 1.46122925, -1.74407392, -0.36548079,  0.31625333]])

In [10]:
# Проверим её на обратимость.
np.linalg.inv(random_matrix)

array([[ 0.78973677,  0.44676743, -0.43446079,  0.19911799],
       [ 0.33227616,  0.14166773, -0.32804629, -0.40580062],
       [ 0.36073677, -0.20112567,  0.28628432,  0.06982325],
       [-1.39960556, -1.51542504,  0.52913673,  0.08479199]])

In [11]:
# Теперь при перемножении исходной на обратную матрицы, должна получиться единичная матрица
random_matrix.dot(np.linalg.inv(random_matrix))

array([[ 1.00000000e+00,  5.99202284e-17, -4.76769616e-17,
         2.19082903e-17],
       [ 1.90187286e-16,  1.00000000e+00,  3.48073126e-17,
        -3.21695371e-17],
       [-2.24382698e-16, -6.60318903e-17,  1.00000000e+00,
        -7.04071107e-17],
       [ 1.98702118e-16,  9.79186545e-17, -1.41952767e-16,
         1.00000000e+00]])

In [12]:
# Признаки ДатаФрейма умножим на  нашу обратимую матрицу random_matrix
features_encoded = features.dot(random_matrix)
features_encoded.head()

,0,1,2,3
0,-84906.963753,-37201.300459,72236.223464,-38123.594283
1,-65064.274208,-28513.525209,55330.893270,-29221.180440
2,-35960.944088,-15759.393377,30575.274401,-16152.068422
3,-71368.805255,-31267.671863,60738.855172,-32038.550316
4,-44685.787141,-19579.713139,38007.668899,-20067.166155


In [13]:
# Обучим модель на новых признаках, сделаем предсказания и посчитаем R2
model.fit(features_encoded, target)
predictions = model.predict(features_encoded)
r2_score(target, predictions)

[-0.93823499  0.01951115  0.01319811 -0.0052701  -0.06635684]


0.42494550286659416

**Ответ:** Умножив признаки на случайную обратимую матрицу, качество линейной регрессии не меняется.

**Обоснование:** Веса меняются под значения изменённой матрицы, так как признаки исходной матрицы и преобразованной выражаются через веса w, и качество линейной регресии остаётся неизменным.

## Алгоритм преобразования

**Алгоритм**

Будем использовать функцию кодирования исходных данных в зашифрованный вид, в которой подаваемая на вход исходная матрица будет умножаться на случайную обратимую матрицу. А для расшифровки данных будем использовать функцию декодирования, в которой подаваемая на вход зашифрованная матрица умножается на обратную нашей случайной, и возвращает исходные данные.

In [14]:
# Создадим ещё раз случайную матрицу
random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))

In [15]:
# Проверим её на обратимость.
np.linalg.inv(random_matrix)

array([[ 0.15109405,  0.35811063, -0.88891339, -0.381049  ],
       [ 0.04414609,  0.70678584, -0.42713079, -0.30895778],
       [ 0.14096296,  0.21002147, -0.43887812, -0.88765762],
       [ 0.67592067, -0.48139036,  0.43219712,  0.39872306]])

In [16]:
# Теперь при перемножении исходной на обратную матрицы, должна получиться единичная матрица
random_matrix.dot(np.linalg.inv(random_matrix))

array([[ 1.00000000e+00,  6.29923834e-17, -3.09667861e-18,
         6.64262802e-17],
       [ 9.14254715e-18,  1.00000000e+00, -8.30032108e-17,
        -1.20013726e-18],
       [ 3.81003342e-17,  1.55882971e-18,  1.00000000e+00,
        -5.44683838e-17],
       [-1.02947850e-18, -3.68919280e-17,  7.22272571e-17,
         1.00000000e+00]])

In [17]:
# Функция шифрования
def coding(data):
    return data.dot(random_matrix)

In [18]:
# Шифруем
encoded_features = coding(features)
encoded_features.head()

,0,1,2,3
0,-83673.409579,41879.838969,26072.355755,10533.020391
1,-64115.221091,32113.252765,19969.587675,8070.150103
2,-35435.083846,17753.681623,11035.465652,4460.113001
3,-70335.910496,35182.769102,21922.130404,8853.031574
4,-44034.729488,22050.368550,13718.349512,5543.799425


In [19]:
# Функция дешифрования
def decoding(data):
    decoded_data = round(abs(data.dot(np.linalg.inv(random_matrix))))
    for i in decoded_data.iloc[:, [0, 3]]:
        decoded_data[i] = decoded_data[i].astype(int)
    decoded_data.columns = features.columns
    return decoded_data

In [20]:
# Дешифруем
decrypted_features = decoding(encoded_features)
decrypted_features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


In [21]:
# Исходные данные
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


Данные полностью восстановлены

**Обоснование**

Формула предсказаний: $ a=Xw $

После умножения исходных данных на случайную матрицу Z, формула будет такой: $ a' = XZw' $ .

Но $ w' = Z^{-1} w $ , поэтому $ a' = XZZ^{-1} w $

$ Z*Z^{-1} $ это единичная матрица Е, убираем её.

Получим итоговую формулу предсказаний: $ a' = Xw = a $ , т.е. предсказания по исходной и закодированной матрице равны.

## Проверка алгоритма

In [22]:
# Создадим модель Линейной регрессии, обучим её, посчитаем метрику R2 для исходных данных
model = LinearRegression()
model.fit(features, target)
r2_score(target, model.predict(features))

0.4249455028666801

In [23]:
# Посчитаем метрику R2 для закодированных данных
model.fit(encoded_features, target)
r2_score(target, model.predict(encoded_features))

0.42494550286668475

**Выводы**
Качество метрики R2 для матрицы признаков до кодирования и для матрицы признаков после кодирования одинаковы. Метрика R2, рассчитанная по написанному нами классу линейной регресии Regression вначале, имела точно такое же значение. Алгоритм верен.

**Общий вывод:**

- Умножив признаки на обратимую случайную матрицу, качество регрессии не меняется, так как Веса меняются под новые значения, (признаки исходной матрицы и преобразованной выражаются через эти веса w).

- Алгоритм шифрования данных справляется для решения нашей задачи.

- Зашифрованные данные можно использовать для предсказаний с помощью линейной регресии.

- Алгоритм дешифрует данные без потерь.

- Так как в алгоритме применён генератор случайной матрицы, она при каждом запуске будет новой, и зашифрованные данные также каждый раз будут новыми. То есть, данные сложно восстановить без кода программы.

- Алгоритм удовлетворяет поставленной задаче Защиты персональных данных клиентов.